# LangGraph Conversational Context

Understanding how LangGraph maintains state across multiple conversation turns.

## Learning Objectives

By the end of this notebook, you will:

1. **Understand state persistence** - How the conversation state accumulates messages across multiple user turns
2. **Recognize context usage** - How the LLM leverages previous messages to understand references like "that amount" or "the EMI you calculated"
3. **Manage conversation state** - How to maintain and pass state between turns to create multi-turn conversations

## 1. Environment Setup

In [1]:
# Core imports
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langchain_google_genai import ChatGoogleGenerativeAI

from dotenv import load_dotenv
from typing import Literal

load_dotenv("../../.env")
print("✅ Environment loaded")

✅ Environment loaded


In [2]:
# Mermaid helper for graph visualization
def render_mermaid(diagram_code, width=400):
    """Render Mermaid diagrams using mermaid.ink service."""
    from IPython.display import Image, display
    import base64
    
    graphbytes = diagram_code.encode('utf-8')
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode('ascii')
    url = f'https://mermaid.ink/img/{base64_string}'
    display(Image(url=url, width=width))

print("✅ Visualization helper defined")

✅ Visualization helper defined


## 2. Define Tools

In [3]:
# Define tools
@tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Convert currency from one type to another.
    
    Use this tool when users need to convert monetary amounts between
    different currencies. Supports USD, EUR, GBP, INR, and JPY.
    """
    exchange_rates = {"USD": 1.0, "EUR": 0.92, "GBP": 0.79, "INR": 83.12, "JPY": 149.50}
    from_currency = from_currency.upper()
    to_currency = to_currency.upper()
    
    if from_currency not in exchange_rates or to_currency not in exchange_rates:
        return f"Error: Unsupported currency"
    
    amount_in_usd = amount / exchange_rates[from_currency]
    converted_amount = amount_in_usd * exchange_rates[to_currency]
    effective_rate = exchange_rates[to_currency] / exchange_rates[from_currency]
    
    return (
        f"Conversion Result:\n"
        f"  {amount:,.2f} {from_currency} = {converted_amount:,.2f} {to_currency}\n"
        f"  Exchange Rate: 1 {from_currency} = {effective_rate:.4f} {to_currency}"
    )

@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.
    
    Use this tool when users want to know their monthly loan payment,
    total repayment amount, or total interest for a loan.
    """
    if principal <= 0 or annual_interest_rate < 0 or tenure_months <= 0:
        return "Error: Invalid input parameters"
    
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    return (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months\n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )

print("✅ Tools defined")

✅ Tools defined


## 3. Initialize LLM and Build Graph

In [4]:
# Initialize LLM and build graph
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=1024
)

tools = [currency_converter, emi_calculator]
llm_with_tools = llm.bind_tools(tools)

def call_llm(state: MessagesState):
    """LLM node: Calls LLM with current messages."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Router: Check if agent wants to use tools."""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

# Build graph
workflow = StateGraph(MessagesState)
workflow.add_node("llm", call_llm)
workflow.add_node("tools", ToolNode(tools))
workflow.add_edge(START, "llm")
workflow.add_conditional_edges("llm", should_continue, {"tools": "tools", END: END})
workflow.add_edge("tools", "llm")

app = workflow.compile()
print("✅ Graph compiled")

✅ Graph compiled


In [5]:
# Visualize graph
mermaid_diagram = app.get_graph().draw_mermaid()
render_mermaid(mermaid_diagram, width=200)

## 4. Multi-Turn Conversation Example

We'll create a conversation where each turn references previous results.

In [6]:
# Initialize conversation state
conversation = {
    "messages": []
}

print("Starting Multi-Turn Conversation")
print("=" * 80)

Starting Multi-Turn Conversation


### Turn 1: Currency Conversion

In [7]:
# Turn 1: Currency conversion
print("\n" + "─" * 80)
print("TURN 1")
print("─" * 80)

user_query_1 = "Convert 50000 USD to INR"
print(f"👤 User: {user_query_1}")

conversation["messages"].append(HumanMessage(content=user_query_1))
conversation = app.invoke(conversation)

print(f"\n🤖 Assistant: {conversation['messages'][-1].content}")
print(f"\n📊 State size: {len(conversation['messages'])} messages")


────────────────────────────────────────────────────────────────────────────────
TURN 1
────────────────────────────────────────────────────────────────────────────────
👤 User: Convert 50000 USD to INR

🤖 Assistant: 50,000 USD is equivalent to 4,156,000 INR.

📊 State size: 4 messages


In [9]:
dict(conversation["messages"][0])

{'content': 'Convert 50000 USD to INR',
 'additional_kwargs': {},
 'response_metadata': {},
 'type': 'human',
 'name': None,
 'id': '8da9aa7d-88c3-4cf2-9d28-181327c4b4ca'}

In [10]:
dict(conversation["messages"][1])

{'content': '',
 'additional_kwargs': {'function_call': {'name': 'currency_converter',
   'arguments': '{"from_currency": "USD", "to_currency": "INR", "amount": 50000}'}},
 'response_metadata': {'finish_reason': 'STOP',
  'model_name': 'gemini-2.0-flash',
  'safety_ratings': [],
  'model_provider': 'google_genai'},
 'type': 'ai',
 'name': None,
 'id': 'lc_run--019c376a-d136-7ca2-bc59-3be9c7468e4f-0',
 'tool_calls': [{'name': 'currency_converter',
   'args': {'from_currency': 'USD', 'to_currency': 'INR', 'amount': 50000},
   'id': 'c24ab0e1-ff18-45fe-be4b-7c44f8a5f8ad',
   'type': 'tool_call'}],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 138,
  'output_tokens': 13,
  'total_tokens': 151,
  'input_token_details': {'cache_read': 0}}}

In [11]:
dict(conversation["messages"][2])

{'content': 'Conversion Result:\n  50,000.00 USD = 4,156,000.00 INR\n  Exchange Rate: 1 USD = 83.1200 INR',
 'additional_kwargs': {},
 'response_metadata': {},
 'type': 'tool',
 'name': 'currency_converter',
 'id': '04df7ce7-1b09-4e3a-947b-086968457151',
 'tool_call_id': 'c24ab0e1-ff18-45fe-be4b-7c44f8a5f8ad',
 'artifact': None,
 'status': 'success'}

In [12]:
dict(conversation["messages"][3])

{'content': '50,000 USD is equivalent to 4,156,000 INR.',
 'additional_kwargs': {},
 'response_metadata': {'finish_reason': 'STOP',
  'model_name': 'gemini-2.0-flash',
  'safety_ratings': [],
  'model_provider': 'google_genai'},
 'type': 'ai',
 'name': None,
 'id': 'lc_run--019c376a-d61e-7e61-a49a-3757d08a3c5e-0',
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 203,
  'output_tokens': 22,
  'total_tokens': 225,
  'input_token_details': {'cache_read': 0}}}

### Turn 2: EMI Calculation Using Previous Result

Notice: "**that INR amount**" references Turn 1's result!

In [ ]:
# Turn 2: EMI calculation referencing previous conversion
print("\n" + "─" * 80)
print("TURN 2")
print("─" * 80)

user_query_2 = "Calculate EMI for that INR amount at 8.5% for 60 months"
print(f"👤 User: {user_query_2}")
print("\n   Note: 'that INR amount' references Turn 1's result!")

conversation["messages"].append(HumanMessage(content=user_query_2))


────────────────────────────────────────────────────────────────────────────────
TURN 2
────────────────────────────────────────────────────────────────────────────────
👤 User: Calculate EMI for that INR amount at 8.5% for 60 months

   Note: 'that INR amount' references Turn 1's result!

🤖 Assistant: The EMI for a loan of 4,156,000 INR at an annual interest rate of 8.5% for 60 months would be 85,266.70 INR. The total payment would be 5,116,002.25 INR, with a total interest of 960,002.25 INR.

📊 State size: 8 messages


In [13]:
dict(conversation["messages"][4])

{'content': 'Calculate EMI for that INR amount at 8.5% for 60 months',
 'additional_kwargs': {},
 'response_metadata': {},
 'type': 'human',
 'name': None,
 'id': '834d08f7-5830-4baa-9970-679bdcb09ee5'}

In [ ]:
conversation = app.invoke(conversation)

print(f"\n🤖 Assistant: {conversation['messages'][-1].content}")
print(f"\n📊 State size: {len(conversation['messages'])} messages")

In [14]:
dict(conversation["messages"][5])

{'content': '',
 'additional_kwargs': {'function_call': {'name': 'emi_calculator',
   'arguments': '{"currency": "INR", "tenure_months": 60, "principal": 4156000, "annual_interest_rate": 8.5}'}},
 'response_metadata': {'finish_reason': 'STOP',
  'model_name': 'gemini-2.0-flash',
  'safety_ratings': [],
  'model_provider': 'google_genai'},
 'type': 'ai',
 'name': None,
 'id': 'lc_run--019c376a-f7af-7a50-91c4-5eb628ed380e-0',
 'tool_calls': [{'name': 'emi_calculator',
   'args': {'currency': 'INR',
    'tenure_months': 60,
    'principal': 4156000,
    'annual_interest_rate': 8.5},
   'id': '619e7e97-5125-46ad-8350-1872a5fe5349',
   'type': 'tool_call'}],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 242,
  'output_tokens': 18,
  'total_tokens': 260,
  'input_token_details': {'cache_read': 0}}}

In [15]:
dict(conversation["messages"][6])

{'content': 'EMI Calculation Result:\n  Loan Amount: 4,156,000.00 INR\n  Interest Rate: 8.5% per annum\n  Tenure: 60 months\n  Monthly EMI: 85,266.70 INR\n  Total Payment: 5,116,002.25 INR\n  Total Interest: 960,002.25 INR',
 'additional_kwargs': {},
 'response_metadata': {},
 'type': 'tool',
 'name': 'emi_calculator',
 'id': '293602b3-60d1-4aca-8c8e-4f33b01d1b34',
 'tool_call_id': '619e7e97-5125-46ad-8350-1872a5fe5349',
 'artifact': None,
 'status': 'success'}

In [16]:
dict(conversation["messages"][7])

{'content': 'The EMI for a loan of 4,156,000 INR at an annual interest rate of 8.5% for 60 months would be 85,266.70 INR. The total payment would be 5,116,002.25 INR, with a total interest of 960,002.25 INR.',
 'additional_kwargs': {},
 'response_metadata': {'finish_reason': 'STOP',
  'model_name': 'gemini-2.0-flash',
  'safety_ratings': [],
  'model_provider': 'google_genai'},
 'type': 'ai',
 'name': None,
 'id': 'lc_run--019c376a-fbcf-7081-b9e8-e6b45ef1240d-0',
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 360,
  'output_tokens': 85,
  'total_tokens': 445,
  'input_token_details': {'cache_read': 0}}}

### Turn 3: Simple Recall (No Tool Needed)

The LLM answers from memory without calling tools!

In [18]:
# Turn 3: Simple recall question (no tool needed)
print("\n" + "─" * 80)
print("TURN 3")
print("─" * 80)

user_query_3 = "What was the monthly EMI you calculated?"
print(f"👤 User: {user_query_3}")
print("\n   Note: Agent should answer from memory, no tool needed!")

conversation["messages"].append(HumanMessage(content=user_query_3))


────────────────────────────────────────────────────────────────────────────────
TURN 3
────────────────────────────────────────────────────────────────────────────────
👤 User: What was the monthly EMI you calculated?

   Note: Agent should answer from memory, no tool needed!


In [19]:
dict(conversation["messages"][8])

{'content': 'What was the monthly EMI you calculated?',
 'additional_kwargs': {},
 'response_metadata': {},
 'type': 'human',
 'name': None,
 'id': None}

In [20]:
conversation = app.invoke(conversation)

print(f"\n🤖 Assistant: {conversation['messages'][-1].content}")
print(f"\n📊 State size: {len(conversation['messages'])} messages")


🤖 Assistant: The monthly EMI calculated was 85,266.70 INR.

📊 State size: 10 messages


In [21]:
dict(conversation["messages"][9])

{'content': 'The monthly EMI calculated was 85,266.70 INR.',
 'additional_kwargs': {},
 'response_metadata': {'finish_reason': 'STOP',
  'model_name': 'gemini-2.0-flash',
  'safety_ratings': [],
  'model_provider': 'google_genai'},
 'type': 'ai',
 'name': None,
 'id': 'lc_run--019c376e-d2cb-78e2-98fa-55ba4c2def23-0',
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 453,
  'output_tokens': 17,
  'total_tokens': 470,
  'input_token_details': {'cache_read': 0}}}

## 5. Examine Complete Conversation State

In [22]:
conversation['messages']

[HumanMessage(content='Convert 50000 USD to INR', additional_kwargs={}, response_metadata={}, id='8da9aa7d-88c3-4cf2-9d28-181327c4b4ca'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'currency_converter', 'arguments': '{"from_currency": "USD", "to_currency": "INR", "amount": 50000}'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019c376a-d136-7ca2-bc59-3be9c7468e4f-0', tool_calls=[{'name': 'currency_converter', 'args': {'from_currency': 'USD', 'to_currency': 'INR', 'amount': 50000}, 'id': 'c24ab0e1-ff18-45fe-be4b-7c44f8a5f8ad', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 138, 'output_tokens': 13, 'total_tokens': 151, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='Conversion Result:\n  50,000.00 USD = 4,156,000.00 INR\n  Exchange Rate: 1 USD = 83.1200 INR', name='currency_converter', id='04df7ce7-1b09-4e3a-947b

In [23]:
# Examine complete conversation state
print("COMPLETE CONVERSATION STATE")
print("=" * 80)

turn_num = 0
for i, msg in enumerate(conversation["messages"], 1):
    if isinstance(msg, HumanMessage):
        turn_num += 1
        print(f"\n{'─' * 80}")
        print(f"TURN {turn_num}")
        print(f"{'─' * 80}")
        print(f"[{i}] 👤 USER: {msg.content[:60]}..." if len(msg.content) > 60 else f"[{i}] 👤 USER: {msg.content}")
        
    elif isinstance(msg, AIMessage):
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            print(f"[{i}] 🤖 AGENT: Calling {msg.tool_calls[0]['name']}")
        else:
            preview = msg.content[:50] + "..." if len(msg.content) > 50 else msg.content
            print(f"[{i}] 🤖 AGENT: {preview}")
            
    elif isinstance(msg, ToolMessage):
        first_line = msg.content.split('\n')[0]
        print(f"[{i}] 🔧 TOOL: {first_line}")

print(f"\n{'=' * 80}")
print(f"Total messages in state: {len(conversation['messages'])}")

COMPLETE CONVERSATION STATE

────────────────────────────────────────────────────────────────────────────────
TURN 1
────────────────────────────────────────────────────────────────────────────────
[1] 👤 USER: Convert 50000 USD to INR
[2] 🤖 AGENT: Calling currency_converter
[3] 🔧 TOOL: Conversion Result:
[4] 🤖 AGENT: 50,000 USD is equivalent to 4,156,000 INR.

────────────────────────────────────────────────────────────────────────────────
TURN 2
────────────────────────────────────────────────────────────────────────────────
[5] 👤 USER: Calculate EMI for that INR amount at 8.5% for 60 months
[6] 🤖 AGENT: Calling emi_calculator
[7] 🔧 TOOL: EMI Calculation Result:
[8] 🤖 AGENT: The EMI for a loan of 4,156,000 INR at an annual i...

────────────────────────────────────────────────────────────────────────────────
TURN 3
────────────────────────────────────────────────────────────────────────────────
[9] 👤 USER: What was the monthly EMI you calculated?
[10] 🤖 AGENT: The monthly EMI calculat

## 6. Second Example: Car Purchase Planning

A more complex multi-turn scenario with what-if questions.

In [24]:
# Second example: Car purchase planning
car_conversation = {"messages": []}

print("SCENARIO: Car Purchase Planning")
print("=" * 80)

SCENARIO: Car Purchase Planning


In [25]:
# Turn 1: Budget conversion
print("\nTurn 1")
print("-" * 40)
query1 = "I have a budget of 2000000 INR for a car. What's that in USD?"
print(f"👤 User: {query1}")

car_conversation["messages"].append(HumanMessage(content=query1))
car_conversation = app.invoke(car_conversation)

print(f"🤖 Assistant: {car_conversation['messages'][-1].content}")


Turn 1
----------------------------------------
👤 User: I have a budget of 2000000 INR for a car. What's that in USD?
🤖 Assistant: 2,000,000 INR is approximately 24,061.60 USD.


In [26]:
# Turn 2: EMI for full budget
print("\nTurn 2")
print("-" * 40)
query2 = "If I take a loan for my full budget at 9% for 5 years, what's my monthly payment?"
print(f"👤 User: {query2}")
print("   (References 'my full budget' = 2,000,000 INR from Turn 1)")

car_conversation["messages"].append(HumanMessage(content=query2))
car_conversation = app.invoke(car_conversation)

print(f"\n🤖 Assistant: {car_conversation['messages'][-1].content}")


Turn 2
----------------------------------------
👤 User: If I take a loan for my full budget at 9% for 5 years, what's my monthly payment?
   (References 'my full budget' = 2,000,000 INR from Turn 1)

🤖 Assistant: The monthly payment would be 41,516.71 USD.


In [27]:
# Turn 3: What-if scenario
print("\nTurn 3")
print("-" * 40)
query3 = "What if I reduce the tenure to 3 years instead?"
print(f"👤 User: {query3}")
print("   (References same loan amount and rate, just different tenure)")

car_conversation["messages"].append(HumanMessage(content=query3))
car_conversation = app.invoke(car_conversation)

print(f"\n🤖 Assistant: {car_conversation['messages'][-1].content}")


Turn 3
----------------------------------------
👤 User: What if I reduce the tenure to 3 years instead?
   (References same loan amount and rate, just different tenure)

🤖 Assistant: With a 3-year tenure, the monthly payment would be 63,599.47 USD.


In [28]:
# Turn 4: Comparison question
print("\nTurn 4")
print("-" * 40)
query4 = "How much more would I pay monthly with the 3-year plan compared to 5 years?"
print(f"👤 User: {query4}")
print("   (Requires comparing results from Turn 2 and Turn 3)")

car_conversation["messages"].append(HumanMessage(content=query4))
car_conversation = app.invoke(car_conversation)

print(f"\n🤖 Assistant: {car_conversation['messages'][-1].content}")
print(f"\n📊 Final state size: {len(car_conversation['messages'])} messages")


Turn 4
----------------------------------------
👤 User: How much more would I pay monthly with the 3-year plan compared to 5 years?
   (Requires comparing results from Turn 2 and Turn 3)

🤖 Assistant: The monthly payment for the 5-year plan is 41,516.71 USD and for the 3-year plan, it's 63,599.47 USD. So, you would pay 22,082.76 USD more monthly with the 3-year plan.

📊 Final state size: 14 messages


## 7. Fresh Conversation (No Context)

Starting a new conversation shows it has no memory of previous ones.

In [29]:
# Starting a fresh conversation (no context from previous)
fresh_conversation = {
    "messages": [
        HumanMessage(content="Convert 500 EUR to GBP")
    ]
}

result_fresh = app.invoke(fresh_conversation)

print("FRESH CONVERSATION (New State)")
print("=" * 80)
print(f"Messages in state: {len(result_fresh['messages'])}")
print(f"\n🤖 Response: {result_fresh['messages'][-1].content}")
print("\n✅ No context from previous car purchase conversation!")
print("   (It doesn't know about 2,000,000 INR or any EMI calculations)")

FRESH CONVERSATION (New State)
Messages in state: 4

🤖 Response: 500 EUR is approximately 429.35 GBP.

✅ No context from previous car purchase conversation!
   (It doesn't know about 2,000,000 INR or any EMI calculations)


## 8. State Growth Visualization

In [30]:
# Visualize state growth
print("STATE GROWTH VISUALIZATION")
print("=" * 80)
print("\nCar Purchase Conversation:")
print()

human_count = sum(1 for m in car_conversation["messages"] if isinstance(m, HumanMessage))
ai_count = sum(1 for m in car_conversation["messages"] if isinstance(m, AIMessage))
tool_count = sum(1 for m in car_conversation["messages"] if isinstance(m, ToolMessage))

print(f"  👤 HumanMessages:  {human_count}")
print(f"  🤖 AIMessages:     {ai_count}")
print(f"  🔧 ToolMessages:   {tool_count}")
print(f"  {'─' * 21}")
print(f"  📊 Total:          {len(car_conversation['messages'])}")

print("\n" + "-" * 80)
print("State growth per turn:")
print("  Turn 1: +1 Human, +2 AI (tool call + response), +1 Tool = 4 messages")
print("  Turn 2: +1 Human, +2 AI, +1 Tool = 4 messages (total: 8)")
print("  Turn 3: +1 Human, +2 AI, +1 Tool = 4 messages (total: 12)")
print("  Turn 4: +1 Human, +1 AI (no tool needed) = 2 messages (total: 14)")

STATE GROWTH VISUALIZATION

Car Purchase Conversation:

  👤 HumanMessages:  4
  🤖 AIMessages:     7
  🔧 ToolMessages:   3
  ─────────────────────
  📊 Total:          14

--------------------------------------------------------------------------------
State growth per turn:
  Turn 1: +1 Human, +2 AI (tool call + response), +1 Tool = 4 messages
  Turn 2: +1 Human, +2 AI, +1 Tool = 4 messages (total: 8)
  Turn 3: +1 Human, +2 AI, +1 Tool = 4 messages (total: 12)
  Turn 4: +1 Human, +1 AI (no tool needed) = 2 messages (total: 14)


## Conclusion

In this notebook, you learned:

✅ **State persistence** - The conversation state accumulates all messages (HumanMessage, AIMessage, ToolMessage) across turns, growing from 4 messages (Turn 1) to 14+ messages (Turn 4+)

✅ **Context usage** - The LLM reads the entire message history to understand references like "that INR amount" (Turn 2), "my full budget" (Turn 2), and "the 3-year plan" (Turn 4), enabling natural multi-turn conversations

✅ **State management** - By manually passing and updating the conversation dictionary between turns, we maintain context; starting a new state creates a fresh conversation with no memory of previous interactions

### Series Complete!

You've now completed the LangGraph tutorial series, covering:
- **Notebooks 01-02**: Setup and basic tool creation
- **Notebooks 03-04**: Building practical tools (currency converter, EMI calculator)
- **Notebook 05**: Graph construction with nodes and edges
- **Notebook 06**: Single tool execution flow
- **Notebook 07**: Parallel execution for independent tasks
- **Notebook 08**: Sequential execution for dependent tasks
- **Notebook 09**: Conversational context and state management

You're now ready to build sophisticated agentic workflows with LangGraph!